<a href="https://colab.research.google.com/github/kuuuun/python_jupyter_notes/blob/main/chapter06_ArrayTransformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chapter06 数组变形记

In [2]:
import numpy as np

## 基础重塑与 -1 大法

> **⚠️ 常见误区** ：  
> `-1` 只能使用一次。你不能写 `reshape(-1, -1)` ，因为这样方程有无数个解，NumPy 会因为无法确定而报错。

In [ ]:
import numpy as np

# 1. 创建一个包含 24 个元素的数组
arr = np.arange(24)
print(f"原始形状: {arr.shape}")

# 2. 变成 4行 6列
matrix = arr.reshape(4, 6)
print(f"\nReshape (4, 6):\n{matrix}")

# 3. 变成 3D 数组 (2块, 3行, 4列)
# 注意：2 * 3 * 4 = 24，必须守恒
tensor = arr.reshape(2, 3, 4)
print(f"\nReshape (2, 3, 4) 的维度: {tensor.ndim}")

# 4. 自动计算 (-1 Trick)
# 我想要 3 行，列数你帮我算
auto_calc = arr.reshape(3, -1)
print(f"\nReshape (3, -1) 自动计算出的形状: {auto_calc.shape}")
# 结果应该是 (3, 8)

原始形状: (24,)

Reshape (4, 6):
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]]

Reshape (2, 3, 4) 的维度: 3

Reshape (3, -1) 自动计算出的形状: (3, 8)


## concatenate：延长管道

In [3]:
a = np.array([[1, 2], [3, 4]]) # (2, 2)
b = np.array([[5, 6]])         # (1, 2)

# 在 axis=0 (垂直方向) 拼接
# 要求：除了拼接轴，其他轴的形状必须完全一致
res = np.concatenate((a, b), axis=0)
print(f"Concatenate 结果形状: {res.shape}") # (3, 2)

Concatenate 结果形状: (3, 2)


## stack：叠盘子

6.2.3 快捷助手：hstack 和 vstack

对于常用的 2D 操作，NumPy 提供了更直观的函数：

vstack (Vertical Stack) ：垂直堆叠，等同于 concatenate(axis=0) 。
hstack (Horizontal Stack) ：水平堆叠，等同于 concatenate(axis=1) 。

In [4]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

# 1. Stack (axis=0) -> 增加最外层维度
# 变成了 (2, 3) 的矩阵
s0 = np.stack((a, b), axis=0)
print(f"Stack axis=0:\n{s0}")
print(f"Shape: {s0.shape}")

# 2. Stack (axis=1) -> 增加列维度
# 变成了 (3, 2) 的矩阵 (相当于把两个向量竖起来并排)
s1 = np.stack((a, b), axis=1)
print(f"\nStack axis=1:\n{s1}")
print(f"Shape: {s1.shape}")

Stack axis=0:
[[1 2 3]
 [4 5 6]]
Shape: (2, 3)

Stack axis=1:
[[1 4]
 [2 5]
 [3 6]]
Shape: (3, 2)


## 分割 (Splitting)：切蛋糕的艺术

有合必有分。 split 是 concatenate 的逆操作。

np.split(ary, indices_or_sections, axis=0)

np.vsplit (垂直切分)

np.hsplit (水平切分)


In [ ]:
import numpy as np

# 模拟一个数据集：10 个样本，每个样本 4 个特征
data = np.arange(40).reshape(10, 4)
print(f"完整数据集形状: {data.shape}")

# 需求：前 7 个做训练集，后 3 个做测试集
# split 接受一个列表作为切分点：[7] 表示在索引 7 处切一刀
train, test = np.split(data, [7], axis=0)

print(f"训练集形状: {train.shape}")
print(f"测试集形状: {test.shape}")

# 也可以等分：将数据分成 2 份
part1, part2 = np.split(data, 2, axis=0)
print(f"等分后形状: {part1.shape}")

完整数据集形状: (10, 4)
训练集形状: (7, 4)
测试集形状: (3, 4)
等分后形状: (5, 4)


### T property

In [7]:
data = np.zeros((100,200,3))
data_t = data.T
print(data.shape)
print(data_t.shape)

(100, 200, 3)
(3, 200, 100)


In [ ]:
# 模拟一张图片 (H=100, W=200, C=3)
img_hwc = np.zeros((100, 200, 3))

# 目标：变成 (C, H, W)
# 原始索引: 0->H, 1->W, 2->C
# 目标索引: 2->C, 0->H, 1->W
# 所以 transpose 的参数是 (2, 0, 1)
img_chw = img_hwc.transpose(2, 0, 1)

print(f"原始形状: {img_hwc.shape}")
print(f"转置后形状: {img_chw.shape}")

原始形状: (100, 200, 3)
转置后形状: (3, 100, 200)


In [ ]:
# 将 axis=2 (Channel) 移动到 axis=0 的位置
img_moved = np.moveaxis(img_hwc, source=2, destination=0)
print(f"Moveaxis 结果: {img_moved.shape}") # (3, 100, 200)

Moveaxis 结果: (3, 100, 200)


## 实战案例：手写数字图像的 Batch 预处理

In [ ]:
import numpy as np

# 1. 模拟读取 3 张图片
# 假设图片是 (28, 28) 的二维数组
rng = np.random.default_rng(42)
img1 = rng.random((28, 28))
img2 = rng.random((28, 28))
img3 = rng.random((28, 28))

print("单张图片形状:", img1.shape)

# 2. 为每张图片增加 Channel 维度
# 现在的深度学习框架通常需要单通道显式存在
# 形状变为 (28, 28, 1)
img1_c = img1[:, :, np.newaxis]
img2_c = img2[:, :, np.newaxis]
img3_c = img3[:, :, np.newaxis]

# 3. 堆叠成 Batch
# 我们希望 Batch 维度在最前面 (axis=0)
# 形状变为 (3, 28, 28, 1)
batch_hwc = np.stack([img1_c, img2_c, img3_c], axis=0)

print(f"堆叠后的 Batch (NHWC): {batch_hwc.shape}")

# 4. 调整维度顺序 (NHWC -> NCHW)
# 当前: 0->N, 1->H, 2->W, 3->C
# 目标: 0->N, 3->C, 1->H, 2->W
batch_nchw = batch_hwc.transpose(0, 3, 1, 2)

print(f"最终输入模型的格式 (NCHW): {batch_nchw.shape}")

# 5. (可选) 展平送入全连接层
# 保持 N 不变，后面拉直
# 3 * 1 * 28 * 28 = 2352
flat_input = batch_nchw.reshape(batch_nchw.shape[0], -1)

print(f"全连接层输入: {flat_input.shape}") # (3, 784)

单张图片形状: (28, 28)
堆叠后的 Batch (NHWC): (3, 28, 28, 1)
最终输入模型的格式 (NCHW): (3, 1, 28, 28)
全连接层输入: (3, 784)
